In [39]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [6]:
page = requests.get("https://weather.com/da-DK/weather/tenday/l/805cf27fe1a64f809e48f19c9ed2b59c419b6cd7b837395f7f6c14881756ca3d")

In [7]:
soup = BeautifulSoup(page.content, 'html.parser')

In [45]:
weekday_day_one = soup.find(id = 'titleIndex1').find('h2')
info_day_1 = soup.find(id = 'titleIndex1').find_all('div')
info_day_1[0]

<div class="DetailsSummary--temperature--1Syw3" data-testid="detailsTemperature"><span class="DetailsSummary--highTempValue--3Oteu" data-testid="TemperatureValue">6°</span><span data-testid="lowTempValue">/<span class="DetailsSummary--lowTempValue--3H-7I" data-testid="TemperatureValue">-1°</span></span></div>

In [33]:
print(weekday_day_one.get_text())
for info in info_day_1:
    print(info.get_text())

søn. 06
6°/-1°
Partly CloudyDelvist skyet
Rain12%
WindNV 13 km/t.


In [48]:
df = pd.DataFrame({'date': [], 'weekday': [], 'day_temp': [], 'night_temp': [], 
                  'description': [], 'rain_%': [],
                  'wind_strength': [], 'wind_direction': []})

In [54]:
for i in range(1,10):
    data = soup.find(id = 'titleIndex{}'.format(i))
    day = data.find('h2').get_text().split('. ')
    info = data.find_all('div')
    temp = info[0].get_text().split('/')
    desc = info[1].get_text()
    rain = re.findall(r'\d+', info[2].get_text())[0]
    wind = info[3].get_text().split(' ')
    df.loc[i - 1] = [day[1], day[0], temp[0], temp[1], desc, rain, wind[1], wind[0]]

In [55]:
df.head(10)

,date,weekday,day_temp,night_temp,description,rain_%,wind_strength,wind_direction
0,06,søn,6°,-1°,Partly CloudyDelvist skyet,12,13,WindNV
1,07,man,8°,-2°,Mostly SunnyMest solrigt,12,12,WindNNV
2,08,tir,7°,-1°,Mostly SunnyMest solrigt,12,7,WindNNV
3,09,ons,6°,1°,Mostly SunnyMest solrigt,12,11,WindØSØ
4,10,tor,4°,1°,Partly CloudyDelvist skyet,8,23,WindSØ
5,11,fre,5°,1°,Mostly SunnyMest solrigt,5,26,WindSØ
6,12,lør,5°,1°,Partly CloudyDelvist skyet,8,24,WindSØ
7,13,søn,6°,1°,Partly CloudyDelvist skyet,7,20,WindØSØ
8,14,man,6°,2°,Partly CloudyDelvist skyet,18,21,WindØSØ


In [57]:
df.to_csv('../Data/cleaned_data.csv', index = False)